# ETL

En esta libreta se busca scrapear la página de Alerta Amber para obtener imágenes y datos de los niños y adolescentes desaparecidos.

Al revisar el código de la página de [Alerta Amber](https://alertaamber.fgr.org.mx/ "Alerta Amber"), nos dimos cuenta que había dos tipos de numeración para los PDF's de los reportes de desaparecidos uno de 4 dígitos que va desde 0000 hasta 9999, y uno de tres dígitos que va desde 000 hasta 999. En este documento escrapeamos ambos, pero al comparar los datos, nos dimos cuenta que se repiten los primeros reportes, esto es 0398 es igual a 398. De todas formas decidimos dejar el código que scrapea los reportes de tres dígitos.

Otro problem que encontramos fue que la información sobre el estado de la alerta es una imagen en el pdf.

(Falta diagrama)

## Extraer 1 (PDF)

Librerías

In [16]:
import os
import requests
import PyPDF2
import pandas as pd
import numpy as np

In [50]:
def scrapeado_pdf(digitos, letra):
    
    # Base URL of the webpage to scrape for PDFs
    base_url_pdf = "https://appalertaamber.fgr.org.mx/Alerta/CreaAlertaPDFPublico?numero_reporte="
    
    # Create folders if they don't exist
    current_directory = os.getcwd()
    data_folder = os.path.join(os.path.dirname(current_directory), "data")
    pdf_folder = os.path.join(data_folder, "pdf")
    
    for folder in [data_folder, pdf_folder]:
        if not os.path.exists(folder):
            os.makedirs(folder)
    
    # Generate links by changing the last 4 digits
    links = [base_url_pdf + str(i).zfill(digitos) for i in range(1, 10000)]
    
    # Iterate over the links and send requests to retrieve PDFs
    for index, pdf_link in enumerate(links, start=1):
        # Sending request for PDF link
        response_pdf = requests.get(pdf_link)
        if response_pdf.status_code == 200:
            # Save the PDF to the pdf folder
            pdf_path = os.path.join(pdf_folder, f"{letra}_{index}.pdf")
            with open(pdf_path, "wb") as pdf_file:
                pdf_file.write(response_pdf.content)
            # print("Downloaded PDF:", pdf_path)
            
            # Read the content of the PDF
            try:
                with open(pdf_path, 'rb') as pdf_file:
                    pdf_reader = PyPDF2.PdfFileReader(pdf_file)
                    text = ""
                    for page_num in range(pdf_reader.numPages):
                        text += pdf_reader.getPage(page_num).extractText()
                    
                    # Check if the string "Publicación no disponible" is present in the text
                    if "Publicación no disponible" in text:
                        os.remove(pdf_path)  # Delete the PDF file
                        # print("Deleted PDF:", pdf_path)
            except PyPDF2.utils.PdfReadError:
                print("Failed to read PDF:", pdf_path)
        else:
            print("Failed to retrieve data for PDF link:", pdf_link)


### Para 4 dígitos

In [8]:
scrapeado_pdf(4, "A")

### Para 3 dígitos

In [51]:
scrapeado_pdf(3, "B")

## Transformar

### Crear Dataframe

In [18]:
# Function to read PDF files from a directory
def read_pdfs_from_directory(directory):
    # List all files in the directory
    files = os.listdir(directory)
    
    # Filter out only PDF files
    pdf_files = [file for file in files if file.lower().endswith(".pdf")]
    
    if not pdf_files:
        print("No PDF files found in the directory.")
        return None
    
    # Create an empty list to store DataFrames for each PDF
    pdf_data_list = []
    
    # Iterate over each PDF file
    for pdf_file in pdf_files:
        pdf_path = os.path.join(directory, pdf_file)
        # print(f"Reading PDF file: {pdf_path}")
        
        # Open the PDF file
        with open(pdf_path, "rb") as file:
            # Create PDF reader object
            pdf_reader = PyPDF2.PdfFileReader(file)
            
            # Get number of pages in the PDF
            num_pages = pdf_reader.numPages
            
            # Initialize text for each page
            pdf_text = ""
            
            # Extract text from each page
            for page_num in range(num_pages):
                page = pdf_reader.getPage(page_num)
                pdf_text += page.extractText()
            
            # Store the text and the last 4 digits of the PDF file name in a DataFrame
            id_file = pdf_file[0:-4]  # Extract the last 4 digits from the file name
            pdf_data_list.append(pd.DataFrame({"Text": [pdf_text], "id_file": [id_file]}))
    
    # Concatenate all DataFrames in the list into a single DataFrame
    pdf_data = pd.concat(pdf_data_list, ignore_index=True)
    
    return pdf_data

In [19]:
# Directory containing PDF files
pdf_directory = "data/pdf"

# Check if the directory exists
if not os.path.exists(pdf_directory):
    print(f"Directory '{pdf_directory}' does not exist.")
else:
    # Read PDFs from the directory
    pdf_dataframe = read_pdfs_from_directory(pdf_directory)
    
    if pdf_dataframe is not None:
        print("\nData from PDFs:")
        print(pdf_dataframe.head())  # Display the first few rows of the DataFrame


Data from PDFs:
                                                Text id_file
0   REPORTE NÚM.: AAMX1896\n FECHA DE ACTIVACIÓN:...  A_1896
1   REPORTE NÚM.: AAMX1725\n FECHA DE ACTIVACIÓN:...  A_1725
2   REPORTE NÚM.: AAMX1758\n FECHA DE ACTIVACIÓN:...  B_1758
3   REPORTE NÚM.: AAMX1084\n FECHA DE ACTIVACIÓN:...  A_1084
4   REPORTE NÚM.: AAMX1416\n FECHA DE ACTIVACIÓN:...  A_1416


### Limpieza

In [56]:
# Separar datos en columnas
pdf_dataframe[['A', 'B']] = pdf_dataframe["Text"].str.split(': ', n=1, expand=True)
pdf_dataframe[['reporte', 'B']] = pdf_dataframe["B"].str.split('\n', n=1, expand=True)
pdf_dataframe[['A', 'B']] = pdf_dataframe["B"].str.split(':', n=1, expand=True)
pdf_dataframe[['fecha_act', 'B']] = pdf_dataframe["B"].str.split('\n', n=1, expand=True)
pdf_dataframe[['nombre', 'B']] = pdf_dataframe["B"].str.split('FECHA DE NACIMIENTO:', n=1, expand=True)
pdf_dataframe[['fecha_nac', 'B']] = pdf_dataframe["B"].str.split('EDAD:', n=1, expand=True)
pdf_dataframe[['edad', 'B']] = pdf_dataframe["B"].str.split('GÉNERO:', n=1, expand=True)
pdf_dataframe[['genero', 'B']] = pdf_dataframe["B"].str.split('FECHA', n=1, expand=True)
pdf_dataframe[['A', 'B']] = pdf_dataframe["B"].str.split(':', n=1, expand=True)
pdf_dataframe[['fecha_hechos', 'B']] = pdf_dataframe["B"].str.split('LUGAR', n=1, expand=True)
pdf_dataframe[['A', 'B']] = pdf_dataframe["B"].str.split(':', n=1, expand=True)
pdf_dataframe[['lugar', 'B']] = pdf_dataframe["B"].str.split('NACIONALIDAD:', n=1, expand=True)
pdf_dataframe[['nacionalidad', 'B']] = pdf_dataframe["B"].str.split('CABELLO:', n=1, expand=True)
pdf_dataframe[['tipo_cabello', 'B']] = pdf_dataframe["B"].str.split('COLOR:', n=1, expand=True)
pdf_dataframe[['color_cabello', 'B']] = pdf_dataframe["B"].str.split('COLOR', n=1, expand=True)
pdf_dataframe[['A', 'B']] = pdf_dataframe["B"].str.split('OJOS:', n=1, expand=True)
pdf_dataframe[['color_ojos', 'B']] = pdf_dataframe["B"].str.split('ESTATURA:', n=1, expand=True)
pdf_dataframe[['estatura', 'B']] = pdf_dataframe["B"].str.split('PESO:', n=1, expand=True)
pdf_dataframe[['peso', 'B']] = pdf_dataframe["B"].str.split('SEÑAS', n=1, expand=True)
pdf_dataframe[['A', 'B']] = pdf_dataframe["B"].str.split('PARTICULARES:', n=1, expand=True)
pdf_dataframe[['senas_part', 'B']] = pdf_dataframe["B"].str.split('RESUMEN', n=1, expand=True)
pdf_dataframe[['A', 'B']] = pdf_dataframe["B"].str.split('HECHOS:', n=1, expand=True)
pdf_dataframe[['resumen_hechos', 'B']] = pdf_dataframe["B"].str.split('RESUMEN', n=1, expand=True)
pdf_dataframe[['senas_part', 'B']] = pdf_dataframe["senas_part"].str.split('Acompañante', n=1, expand=True)
pdf_dataframe[['A', 'acompanante']] = pdf_dataframe["B"].str.split('NOMBRE:', n=1, expand=True)
pdf_dataframe[['acompanante', 'B']] = pdf_dataframe["acompanante"].str.split('SEÑAS', n=1, expand=True)
pdf_dataframe[['A', 'acompanante_sp']] = pdf_dataframe["B"].str.split('PARTICULARES:', n=1, expand=True)
pdf_dataframe[['senas_part', 'B']] = pdf_dataframe["senas_part"].str.split('Sospechoso', n=1, expand=True)
pdf_dataframe[['A', 'sospechoso']] = pdf_dataframe["B"].str.split('NOMBRE:', n=1, expand=True)
pdf_dataframe[['sospechoso', 'B']] = pdf_dataframe["sospechoso"].str.split('SEÑAS', n=1, expand=True)
pdf_dataframe[['A', 'sospechoso_sp']] = pdf_dataframe["B"].str.split('PARTICULARES:', n=1, expand=True)

# Quitar '\n'
# Columns to process
columns_to_process = ['nombre', 'edad', 'fecha_nac', 'genero', 'fecha_hechos', 'lugar', 
                      'nacionalidad', 'tipo_cabello', 'color_cabello', 'color_ojos', 
                      'estatura', 'peso', 'senas_part', 'acompanante', 'acompanante_sp', 
                      'sospechoso', 'sospechoso_sp', 'resumen_hechos']

# Iterate over columns and replace '\n'
for column in columns_to_process:
    pdf_dataframe[column] = pdf_dataframe[column].str.replace(r'\n', ' ', regex=True)

pdf_dataframe = pdf_dataframe.map(lambda x: x.strip() if isinstance(x, str) else x)

# Renombrar y quitar duplicados
df1 = pdf_dataframe[['id_file', 'reporte', 'fecha_act', 'nombre', 'fecha_nac', 'edad', 'genero', 'fecha_hechos',
                    'lugar', 'nacionalidad', 'tipo_cabello', 'color_cabello', 'color_ojos', 'estatura',
                    'peso', 'senas_part', 
                     'acompanante', 'acompanante_sp', 'sospechoso', 'sospechoso_sp',
                     'resumen_hechos']]

df1 = df1.drop_duplicates(subset=["reporte"], keep='first') 

df1


,id_file,reporte,fecha_act,nombre,fecha_nac,edad,genero,fecha_hechos,lugar,nacionalidad,...,color_cabello,color_ojos,estatura,peso,senas_part,acompanante,acompanante_sp,sospechoso,sospechoso_sp,resumen_hechos
0,A_1896,AAMX1896,01/04/2024,ELISA LANDEROS N,05/06/2016,7 años,Femenino,30/03/2024,"TECAMAC, ESTADO DE MEXICO",MEXICANA,...,Castaño_Obscuro,Castaño_Obscuros,1.2 m,22 kg.,NINGUNA.,MYRIAM LANDEROS N,NINGUNA.,None,None,"EL 30 DE MARZO DE 2024, LA NIÑA ELISA LANDEROS..."
1,A_1725,AAMX1725,02/06/2023,ARIANA ROMINA PÉREZ,11/04/2019,3 años,Femenino,12/06/2022,"CUERNAVACA, MORELOS",MEXICANA,...,Negro,Castaño_Obscuros,1 m,27 kg.,HOYUELOS EN MEJILLA.,ANDRÉS PÉREZ GUZMÁN,SIN DATOS.,None,None,"EL 12 DE JUNIO DE 2022, LA NIÑA ARIANA ROMINA ..."
2,B_1758,AAMX1758,09/08/2023,PERLA GARCÍA CRUZ,19/01/2016,7 años,Femenino,04/08/2023,"SAN JUAN BAUTISTA TUXTEPEC, OAXACA",MEXICANA,...,Castaño_Obscuro,Castaño_Claros,Sin dato,Sin dato,"VESTIMENTA: BLUSA COLOR BLANCO, PANTALÓN DE ME...",None,None,None,None,"EL 04 DE AGOSTO DE 2023, LA NIÑA PERLA GARCÍA ..."
3,A_1084,AAMX1084,14/01/2019,ROSA IRMA LETICIA LÓPEZ,14/05/2014,4 años,Femenino,13/01/2019,"UMAN, YUCATÁN",MEXICANA,...,Castaño_Claro,Castaño_Obscuros,1 m,18 kg.,MANCHA DE NACIMIENTO EN EL ESTÓMAGO.,ERNESTO LÓPEZ GUZMÁN,Sin dato,None,None,"EL 13 DE ENERO DE 2019, LA NIÑA ROSA IRMA LETI..."
4,A_1416,AAMX1416,14/01/2022,JOSELIN GARIBAY MATEOS,26/09/2006,15 años,Femenino,12/01/2022,"ZACATEPEC, MORELOS",MEXICANA,...,Castaño_Obscuro,Castaño_Claros,1.56 m,53 kg.,PADECE ESQUIZOFRENIA.,None,None,None,None,"EL 12 DE ENERO DE 2022, LA ADOLESCENTE JOSELIN..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2432,B_570,AAMX570,07/10/2015,N N IBARRA RODRIGUEZ,13/06/2015,3 meses,Femenino,07/10/2015,CHIHUAHUA,MEXICANA,...,Castaño_Obscuro,Castaño_Claros,0.8 m,Sin dato,Sin dato,None,None,None,None,"EL 7 DE OCTUBRE DE 2015, LA C. MARIA ALICIA RO..."
2457,A_1624,AAMX1624,10/12/2022,JESÚS ALEJANDRO FERNÁNDEZ,09/08/2006,16 años,Masculino,09/12/2022,"TOLUCA, ESTADO DE MEXICO",MEXICANA,...,Negro,Castaño_Claros,1.68 m,55 kg.,"LUNAR EN LA NARIZ LADO DERECHO, LUNAR EN EL LA...",None,None,None,None,"EL 09 DE DICIEMBRE DE 2022, EL ADOLESCENTE JES..."
2459,A_1017,AAMX1017,02/08/2018,NEYLIN YULISA HERNÁNDEZ,16/10/2005,12 años,Femenino,30/07/2018,"CUAUHTEMOC, CIUDAD DE MÉXICO",GUATEMALTECA,...,Negro,Castaño_Obscuros,1.6 m,Sin dato,Sin dato,None,None,ERICK DANILO ORTEGA PÉREZ,SOSPECHOSO: \tX NOMBRE:\tERICK DANILO ORTEGA P...,"EL 31 DE JULIO DE 2018, LA ADOLESCENTE NEYLIN ..."
2461,A_710,AAMX710,22/10/2016,MAIIA ELIZABETH HARO,01/10/2016,21 dias,Femenino,22/10/2016,JALISCO,MEXICANA,...,Negro,Grises,0.51 m,3 kg.,Sin dato,None,None,None,None,"EL 22 DE OCTUBRE DE 2016, A LA 01:00 APROXIMAD..."


### Revisar y filtrar qué imágenes queremos antes de descargar

In [57]:
chiapas = df1[df1['lugar'] == "CHIAPAS"]
chiapas

,id_file,reporte,fecha_act,nombre,fecha_nac,edad,genero,fecha_hechos,lugar,nacionalidad,...,color_cabello,color_ojos,estatura,peso,senas_part,acompanante,acompanante_sp,sospechoso,sospechoso_sp,resumen_hechos
26,B_754,AAMX754,10/02/2017,LUIS GUSTAVO LOPEZ SANCHEZ,10/11/2016,2 meses,Masculino,06/02/2017,CHIAPAS,MEXICANA,...,Negro,Castaño_Obscuros,0.45 m,6 kg.,LUNAR EN LA ZONA DE LA COSTILLA DEL LADO DERECHO,None,None,None,None,"EL 06 DE FEBRERO DE 2017, EL BEBÉ LUIS GUSTAVO..."
664,B_696,AAMX696,18/09/2016,DANIEL EZEQUIEL BALCAZAR,17/07/2016,2 meses,Masculino,16/09/2016,CHIAPAS,MEXICANA,...,Sin_Cabello,Castaño_Obscuros,0.5 m,4 kg.,S/D,None,None,ADRIANA,S/D,"EL 16 DE SEPTIEMBRE DE 2016, ÉL BEBE DANIEL EZ..."
1163,B_828,AAMX828,23/05/2017,SURI DARIANA MOLANO,15/07/2003,13 años,Femenino,20/05/2017,CHIAPAS,MEXICANA,...,Castaño_Claro,Castaño_Obscuros,1.5 m,55 kg.,Sin dato,FABIAN REYES PEREZ,Sin dato,None,None,EL 20 DE MAYO DE 2017 SURI DARIANA MOLANO VILL...
1256,A_801,AAMX801,20/04/2017,BRYAN JAVIER RAMIREZ,29/11/2010,6 años,Masculino,21/03/2017,CHIAPAS,MEXICANA,...,Negro,Castaño_Obscuros,1.1 m,19 kg.,CICATRIZ EN LA FRENTE CERCA DE LA CEJA IZQUIERDA,None,None,None,None,"EL NIÑO BRYAN JAVIER RAMIREZ MORALES, FUE VIST..."
1961,A_626,AAMX626,11/03/2016,OSCAR OBED GUILLEN ALBORES,20/03/2015,11 meses,Masculino,10/03/2016,CHIAPAS,MEXICANA,...,Negro,Castaño_Obscuros,0.98 m,9 kg.,Sin dato,KARLA CRUZ HERNANDEZ,Sin dato,None,None,"EL 10 DE MARZO DE 2016, EL BEBÉ OSCAR OBED GUI..."


In [58]:
len(chiapas)

5

In [59]:
len(df1)

1268

### Crear lista para descargar imágenes

In [60]:
df2 = pd.DataFrame(chiapas["id_file"])
df2[['digitos', 'num']] = df2["id_file"].str.split('_', n=1, expand=True)

df_A = df2[df2['digitos'] == "A"]
df_B = df2[df2['digitos'] == "B"]

list_4_digits = df_A['num'].tolist()
list_3_digits = df_B['num'].tolist()

## Extraer 2 (Imágenes)

In [66]:
# Function to download images for a list of numbers
def download_images_for_numbers(numbers, images_folder, digits):
    # Create images folder if it doesn't exist
    current_directory = os.getcwd()
    data_folder = os.path.join(os.path.dirname(current_directory), "data")
    images_folder_path = os.path.join(data_folder, images_folder)
    
    if not os.path.exists(images_folder_path):
        os.makedirs(images_folder_path)
    
    # Iterate over the list of numbers
    for number in numbers:
        # Generate the image link for the current number
        image_link = base_url_image + str(number).zfill(digits)
        
        # Sending request for image link
        response_image = requests.get(image_link)
        if response_image.status_code == 200:
            # Save the image to the images folder
            image_path = os.path.join(images_folder_path, f"image_{number}.jpg")
            with open(image_path, "wb") as img_file:
                img_file.write(response_image.content)
            # print("Downloaded image:", image_path)
        else:
            print("Failed to retrieve data for image link:", image_link)


### Para 3 dígitos

In [73]:
# List of numbers
numbers_to_download = list_3_digits + [398]

# Folder to save images
images_folder = "images"

# Download images for the specified numbers
download_images_for_numbers(numbers_to_download, images_folder, 3)

### Para 4 dígitos

In [74]:
# List of numbers
numbers_to_download = list_4_digits

# Folder to save images
images_folder = "images"

# Download images for the specified numbers
download_images_for_numbers(numbers_to_download, images_folder, 4)

## Cargar

### Subir imágenes de entrenamiento a S3

In [86]:
# Abres un cliente de S3
import boto3
session = boto3.Session(
    profile_name='arquitectura',
region_name='us-east-1'
)
s3 = session.client('s3')

In [87]:
BUCKET_NAME = "itam-proyecto-saraluz"

In [97]:
#Prueba con un archivo (el archivo para test positivo)
s3.upload_file(Filename="./../data/images/image_696.jpg", Bucket=BUCKET_NAME, Key="image_696.jpg")

In [93]:
# List of image names
image_names = list_4_digits + list_3_digits

In [96]:
# Loop through each image name and upload to S3
for image_name in image_names:
    file_path = f"./../data/images/image_{image_name}.jpg"
    s3_key = f"image_{image_name}.jpg"
    s3.upload_file(Filename=file_path, Bucket=BUCKET_NAME, Key=s3_key)

### Subir imágenes de test a S3

In [95]:
image_names

['801', '626', '754', '696', '828']